In [1]:
#GDD calculation for Soy from class SoyGDD. 
import pandas as pd

def fahrenheit_to_celsius(temp_f):
    return (temp_f - 32) * 5 / 9

def celsius_to_fahrenheit(temp_c):
    return (temp_c * 9 / 5) + 32

class SoyGDD:
    def __init__(self, daily_temp_df, time_column, min_temp_column, max_temp_column, celsius=True):
        self.daily_temp_df = daily_temp_df
        self.time_column = time_column
        self.min_temp_column = min_temp_column
        self.max_temp_column = max_temp_column
        self.celsius = celsius

        # Ensure the time column is of datetime type
        self.daily_temp_df[self.time_column] = pd.to_datetime(self.daily_temp_df[self.time_column])

        # If initial data is in Celsius, optionally convert it to Fahrenheit for uniform GDD calculation
        if self.celsius:
            self.daily_temp_df[self.min_temp_column] = self.daily_temp_df[self.min_temp_column].apply(celsius_to_fahrenheit)
            self.daily_temp_df[self.max_temp_column] = self.daily_temp_df[self.max_temp_column].apply(celsius_to_fahrenheit)
            self.celsius = False  # Now all data is in Fahrenheit

    def soy_gdd(self, start_date, end_date):
        start_date = pd.Timestamp(start_date)
        end_date = pd.Timestamp(end_date)

        date_mask = (self.daily_temp_df[self.time_column] >= start_date) & (self.daily_temp_df[self.time_column] <= end_date)
        masked_data = self.daily_temp_df.loc[date_mask].copy()

        gdd_values = []
        cumulative_gdd = 0

        for index, row in masked_data.iterrows():
            min_temp = max(row[self.min_temp_column], 50)  # Fahrenheit base
            max_temp = min(row[self.max_temp_column], 86)  # Fahrenheit upper limit

            mean_temp = (min_temp + max_temp) / 2
            base_temp = 50  # Fahrenheit base temperature for GDD calculation
            gdd = max(mean_temp - base_temp, 0)

            gdd_values.append(gdd)
            cumulative_gdd += gdd

        masked_data['GDD'] = gdd_values
        masked_data['Cumulative_GDD'] = masked_data['GDD'].cumsum()

        return masked_data

# This modified class converts temperatures to Fahrenheit if they start in Celsius, then computes and returns GDD in Fahrenheit.


In [ ]:
Im

In [8]:
import pandas as pd
StPaul_GEMS = pd.read_csv("st_paul_gems_2023_2024.csv", on_bad_lines='skip')

In [9]:
StPaul_GEMS_2024 = SoyGDD(StPaul_GEMS, time_column='datetime', min_temp_column='min_temp', max_temp_column='max_temp')

In [12]:
#Using year 2024 start date and end date for mine gdd
start_date = pd.to_datetime("6-4-2024").date()
end_date = pd.to_datetime("11-3-2024").date()

In [13]:
Soy_gdd_StPaulGEMS2024 = StPaul_GEMS_2024.soy_gdd(start_date, end_date)
print(Soy_gdd_StPaulGEMS2024)

     clouds   datetime  dewpt  dhi  dni  ghi  max_dhi  max_dni  max_ghi  \
520      50 2024-06-04   17.8   58  464  355      121      920      963   
521      36 2024-06-05   10.1   58  464  356      121      920      963   
522      57 2024-06-06    7.3   58  465  356      121      919      964   
523      29 2024-06-07    6.4   58  465  357      121      919      964   
524      55 2024-06-08    8.9   58  465  357      121      919      965   
..      ...        ...    ...  ...  ...  ...      ...      ...      ...   
668      97 2024-10-30    9.2   30  260  128       96      805      505   
669      93 2024-10-31    2.0   30  257  126       96      802      499   
670      42 2024-11-01   -0.6   30  255  124       96      800      494   
671      62 2024-11-02    2.8   29  253  122       95      798      488   
672      98 2024-11-03    6.5   28  241  115       95      796      483   

     max_temp  ...  t_solar_rad  temp          ts  wind_dir  wind_gust_spd  \
520     84.92  ...   